### Connect to Gremlin Server

In [2]:
import os
import sys
from pathlib import Path

import nest_asyncio
nest_asyncio.apply()

container_src_path = Path('/app/src/')
local_src_path = Path(Path.cwd(), 'src/')

# see if this src path exists.
# if it does, we are in a container.
# if not, we are in local.
if not container_src_path.exists():
    src_path = local_src_path
else:
    src_path = container_src_path

src_path_str = str(src_path)
if src_path_str not in sys.path:
    sys.path.insert(0, src_path_str)


from gremlin_python import statics
from gremlin_python.process.traversal import T, Direction
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.process.graph_traversal import GraphTraversalSource
from gremlin_python.process.graph_traversal import __

from graph.base import g
from ipycytoscape_graph_visualization import visualize_graph

from dotenv import load_dotenv

load_dotenv()

from concurrent.futures import ThreadPoolExecutor
from contextlib import contextmanager
from queue import Queue

# Queue to hold statements
statement_queue = Queue()

# Function to add statement to queue
def add_to_queue(statement):
    statement_queue.put(statement)

@contextmanager
def execute_gremlin_statements():
    try:
        yield add_to_queue
    finally:
        statements = []
        while not statement_queue.empty():
            statements.append(statement_queue.get())
        
        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(statement) for statement in statements]
            results = [future.result() for future in futures]

        for result in results:
            print(result)

# test connection to gremlin server
g.V().limit(1).toList()

[v[4120]]

### Get Networkx Graph

In [3]:
import networkx as nx

from models.base import SessionLocal
from models.bitcoin_data import Block, Tx, Address, Input, Output
from graph.base import g
from graph_analyze import GraphAnalyzer


analyzer = GraphAnalyzer(g, SessionLocal)

# interesting_addr = '12higDjoCCNXSA95xZMWUdPvXNmkAduhWv'
# interesting_addr = '12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S'
interesting_addr = '1BBz9Z15YpELQ4QP5sEKb1SwxkcmPb5TMs'

with SessionLocal() as session:
    address = session.query(Address).filter_by(addr=interesting_addr).first()
    
if not address:
    print(f"address {interesting_addr} not found")
    sys.exit(1)

print(f"id of address {address.addr:4}: {address.id}")

my_hist = analyzer.get_address_history(interesting_addr)
graph = analyzer.traversal_to_networkx(my_hist, include_data=True)

print(my_hist)
print(graph)

# dump graph to gexf file
addr_hist_graph_path = Path('/', 'app', 'addr_hist_graph.gexf')
nx.write_gexf(graph, addr_hist_graph_path)

id of address 1BBz9Z15YpELQ4QP5sEKb1SwxkcmPb5TMs: 504
[['withStrategies', OptionsStrategy], ['withStrategies', OptionsStrategy]][['V'], ['has', 'address', 504], ['repeat', [['in', 'sent']]], ['emit'], ['project', 'vertex', 'edges'], ['by', [['elementMap']]], ['by', [['bothE'], ['elementMap'], ['fold']]]]
DiGraph with 14 nodes and 14 edges


In [5]:
from ipycytoscape_graph_visualization import visualize_graph

display(visualize_graph(graph, layout='dagre'))

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeDimensionsIncludeLabels': True, 'rankDir': 'LR'}, cyto…

### Find Coin Sources

In [2]:
def get_sources(g, node_name):
    source_record = {}
    def traverse_sources(vertex, fraction=1.0):
        # Traverse incoming edges
        in_edges = g.V(vertex).inE('sent').toList()
        for edge in in_edges:
            sender = edge.outV().next()
            amount_from_sender = edge['value']
            if 'marked' in sender.properties:
                # Add record if sender is marked
                if sender['name'] in source_record:
                    source_record[sender['name']] += amount_from_sender * fraction
                else:
                    source_record[sender['name']] = amount_from_sender * fraction
            # Recursive case
            if g.V(sender).inE('sent').hasNext():
                sender_total_received = sum(e['value'] for e in g.V(sender).inE('sent').toList())
                amount_fraction = (amount_from_sender / sender_total_received) * fraction
                traverse_sources(sender, amount_fraction)

    # Start traversal
    traverse_sources(g.V().has('name', node_name).next())
    return source_record

print("Sources for Bob:")
print(get_sources(g, 'Bob'))

Sources for Bob:


RuntimeError: Cannot run the event loop while another loop is running

### Execute Some Queries

In [ ]:
from concurrent.futures import ThreadPoolExecutor

# def add_vertex():
#     return g.addV('Address').property('addr', 'some_address').next()

# with ThreadPoolExecutor() as executor:
#     future = executor.submit(add_vertex)
#     result = future.result()
    # print(result)
    
with execute_gremlin_statements() as add_statement:
    add_statement(lambda: g.addV('Address').property('addr', 'some_address').next())

v[4288]
